
This script monitors feature drift between training and current datasets using EvidentlyAI.
It compares input distributions without involving a predictive model and generates an HTML report 
for visualizing data drift over time.

Main steps:
- Load reference and current datasets
- Run Evidently's DataDriftPreset
- Save the drift analysis as an HTML file


In [12]:
import pandas as pd
import os
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from IPython.display import display  # For Jupyter Notebook
import pprint

def monitor_feature_drift(reference_data_path, current_data_path, output_html_path):
    """
    Compare feature distributions between training and current datasets using EvidentlyAI,
    generate a clean HTML report, and print a drift summary in Jupyter Notebook.
    """

    # Load data
    reference_data = pd.read_csv(reference_data_path)
    current_data = pd.read_csv(current_data_path)

    # Only compare feature distributions, no model involved
    drift_report = Report(metrics=[
        DataDriftPreset()
    ])

    drift_report.run(
        reference_data=reference_data,
        current_data=current_data
    )

    # Save HTML report
    os.makedirs(os.path.dirname(output_html_path), exist_ok=True)
    drift_report.save_html(output_html_path)
    print(f"✅ Feature drift report saved to {output_html_path}")

    # --- 🛠️ Extract drift info ---
    result = drift_report.as_dict()

    # Debug: print the structure (only need once)
    pprint.pprint(result)

    # Correctly extract drift info from the second metric
    drifted_columns_info = result['metrics'][1]['result']['drift_by_columns']

    # Build DataFrame
    drift_df = pd.DataFrame([
        {
            'Feature Name': feature_name,
            'Drift Detected': feature_info['drift_detected'],
            'Drift Score (p-value)': feature_info.get('drift_score', None),
            'Test Used': feature_info.get('stattest_name', None),
            'Threshold': feature_info.get('stattest_threshold', None)
        }
        for feature_name, feature_info in drifted_columns_info.items()
    ])

    # Display the full drift info
    display(drift_df)

    # Optionally: Only features with detected drift
    drifted_features_df = drift_df[drift_df['Drift Detected'] == True]
    print("\n🔎 Features with Drift Detected:")
    display(drifted_features_df)

    # Summary
    n_drifted_features = result['metrics'][0]['result']['number_of_drifted_columns']
    share_drifted_features = result['metrics'][0]['result']['share_of_drifted_columns']

    print(f"\n📊 Drift Summary:")
    print(f"Drifted features: {n_drifted_features}")
    print(f"Share of drifted features: {share_drifted_features:.2%}")


In [13]:
# Outside the function: Call it
monitor_feature_drift(
    reference_data_path="/Users/qianzhao/Desktop/Enterprise/formal version/modeling/X_train_final.csv",
    current_data_path="/Users/qianzhao/Desktop/Enterprise/formal version/drift_fairness/simulated_current.csv",
    output_html_path="/Users/qianzhao/Desktop/Enterprise/formal version/drift_fairness/feature_drift_report.html"
)

✅ Feature drift report saved to /Users/qianzhao/Desktop/Enterprise/formal version/drift_fairness/feature_drift_report.html
{'metrics': [{'metric': 'DatasetDriftMetric',
              'result': {'dataset_drift': False,
                         'drift_share': 0.5,
                         'number_of_columns': 22,
                         'number_of_drifted_columns': 1,
                         'share_of_drifted_columns': 0.045454545454545456}},
             {'metric': 'DataDriftTable',
              'result': {'current_fi': None,
                         'dataset_drift': False,
                         'drift_by_columns': {'Age': {'column_name': 'Age',
                                                      'column_type': 'num',
                                                      'current': {'small_distribution': {'x': [15.08974994855543,
                                                                                               19.967231146744748,
                                    

,Feature Name,Drift Detected,Drift Score (p-value),Test Used,Threshold
0,Age,False,3.494809e-01,K-S p_value,0.05
1,Country,False,8.308791e-01,chi-square p_value,0.05
2,Country_grouped,False,5.472024e-01,chi-square p_value,0.05
3,Gender,True,3.423331e-11,chi-square p_value,0.05
4,benefits,False,1.200126e-01,chi-square p_value,0.05
5,care_options,False,1.290178e-01,chi-square p_value,0.05
6,coworkers,False,6.513457e-01,chi-square p_value,0.05
7,family_history,False,4.338445e-01,Z-test p_value,0.05
8,leave,False,9.984951e-01,chi-square p_value,0.05
9,mental_health_consequence,False,2.342071e-01,chi-square p_value,0.05



🔎 Features with Drift Detected:


,Feature Name,Drift Detected,Drift Score (p-value),Test Used,Threshold
3,Gender,True,3.423331e-11,chi-square p_value,0.05



📊 Drift Summary:
Drifted features: 1
Share of drifted features: 4.55%


**Insights**
- A total of 22 features were evaluated for drift detection.

- No overall dataset drift was observed — the overall feature distributions remain stable.

- Only 1 feature exhibited significant drift, representing 4.55% of the total features.

- The feature that drifted:

  - Gender

    - p-value ≈ 3.42e-11, far below the threshold of 0.05

    - Statistical test used: Chi-square test

    - This indicates a highly significant change in the distribution of Gender in the current data compared to the training data.

- Other features such as Age, Country_grouped, remote_work, etc., did not show significant drift (p-value > 0.05).

**Key Takeaways**

- Overall data stability: The dataset remains stable overall, suggesting that the model can likely continue performing well without immediate retraining.
- Monitor the Gender feature: The significant drift in the Gender feature requires attention, as it may impact model fairness or prediction reliability.
- Consider partial retraining: If Gender is an important feature for the model, it is advisable to consider partial retraining or model adjustments to mitigate any potential bias.
- Continue regular monitoring: Even though only one feature drifted this time, continuous monitoring is recommended, especially for demographic features like Age and Country_grouped.
- Communicate findings to stakeholders: It is important to transparently report the observed drift in Gender to stakeholders (e.g., product managers, HR teams) to assess any necessary business adjustments.
